In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext dotenv

%dotenv ../../env/.env

import warnings
import utils
from pandas import Timedelta
import time
from utils import logger, sql_to_df, run_sql, session, conn
import pandas as pd
warnings.filterwarnings('ignore')

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

In [ ]:
staging_tables = [
    'stg_rate_sheet_daily',
    'stg_warehouse_metering_history',
    'stg_access_history',
    'stg_grants_to_roles',
    'stg_query_history',
    'stg_remaining_balance_daily',
    'stg_sessions',
    'stg_storage_usage',
    'stg_table_storage_metrics',
    'stg_usage_in_currency_daily',
    'stg_users',
    'stg_views',
    'stg_warehouse_events_history',
    'stg_warehouse_load_history',
    'target_role_grants',
    'warehouse_grants',
    'warehouse_params',
    'warehouse_usage_granted',
    'warehouses',
    'DAILY_RATES'
]

# to update from staging, dbt seed && dbt run --exclude staging daily_rates

In [ ]:
run_sql("alter warehouse demo set warehouse_size=medium")

In [ ]:
# df = sql_to_df("show tables in schema knot_share.optiml")
# df.name.to_list()

In [ ]:
target_schema = "share_copy.knot"
for t in df.name.to_list():
    sql = f"create or replace table {target_schema}.{t} as select * from knot_share.optiml.{t}"
    run_sql(sql)

In [ ]:
import concurrent.futures

target_schema = "share_copy.knot"

def copy_table(table):
    sql = f"create or replace table {target_schema}.{table} as select * from knot_share.optiml.{table}"
    run_sql(sql)

# List of arguments
arguments = staging_tables # df.name.to_list()

# Create a ThreadPoolExecutor with the desired number of workers
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    # Submit the tasks to the executor
    futures = [executor.submit(copy_table, arg) for arg in arguments]

    # Wait for the tasks to complete
    # concurrent.futures.wait(futures)
    # Process the completed futures
    for future in concurrent.futures.as_completed(futures):
        try:
            # Retrieve the result from the completed future
            result = future.result()
        except Exception as e:
            # Handle the exception or failure
            print(f"Exception occurred: {e}")

In [ ]:
run_sql("create or replace schema share_copy.knot_monitoring clone share_copy.knot;")